In [35]:
# %%
import pandas as pd

data = pd.read_csv('./datasets/mod_03_topic_05_weather_data.csv.gz')


In [36]:
data.info()
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

(145460, 23)

In [37]:
data.isna().mean().sort_values(ascending=False)


Sunshine         0.480098
Evaporation      0.431665
Cloud3pm         0.408071
Cloud9am         0.384216
Pressure9am      0.103568
Pressure3pm      0.103314
WindDir9am       0.072639
WindGustDir      0.070989
WindGustSpeed    0.070555
Humidity3pm      0.030984
WindDir3pm       0.029066
Temp3pm          0.024811
RainTomorrow     0.022460
Rainfall         0.022419
RainToday        0.022419
WindSpeed3pm     0.021050
Humidity9am      0.018246
Temp9am          0.012148
WindSpeed9am     0.012148
MinTemp          0.010209
MaxTemp          0.008669
Location         0.000000
Date             0.000000
dtype: float64

In [38]:
data = data[data.columns[data.isna().mean().lt(0.35)]]

#Remove rows from the dataset where 'RainTomorrow' column has NaN values
data = data.dropna(subset=['RainTomorrow'])

In [39]:
import numpy as np

data_num = data.select_dtypes(include=np.number)
data_cat = data.select_dtypes(include='object')


In [40]:
data_cat['Date'] = pd.to_datetime(data_cat['Date'])
data_cat[['Year', 'Month']] = data_cat['Date'].apply(
    lambda x: pd.Series([x.year, x.month]))

data_num['Year'] = data_cat['Year']
data_cat['Month'] = data_cat['Month'].astype(str)

data_cat.drop('Date', axis=1, inplace=True)
data_cat.drop('Year', axis=1, inplace=True)

In [41]:
max_year = data_num['Year'].max()

train_index = data_num['Year'] < max_year
test_index = data_num['Year'] == max_year

X_train_num = data_num[train_index].drop(columns='Year')
X_train_cat = data_cat[train_index]
y_train = data['RainTomorrow'][train_index]

X_test_num = data_num[test_index].drop(columns='Year')
X_test_cat = data_cat[test_index]
y_test = data['RainTomorrow'][test_index]

In [42]:
from sklearn.impute import SimpleImputer

num_imputer = SimpleImputer().set_output(transform='pandas')

X_train_num = num_imputer.fit_transform(X_train_num)
X_test_num = num_imputer.transform(X_test_num)


In [43]:
cat_imputer = SimpleImputer(strategy='most_frequent').set_output(transform='pandas')

X_train_cat = cat_imputer.fit_transform(X_train_cat)
X_test_cat = cat_imputer.transform(X_test_cat)


In [44]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().set_output(transform='pandas')

X_train_num = scaler.fit_transform(X_train_num)
X_test_num = scaler.transform(X_test_num)

In [45]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(
    drop='if_binary', 
    sparse_output=False).set_output(transform='pandas')

X_train_cat = encoder.fit_transform(X_train_cat)
X_test_cat = encoder.transform(X_test_cat)

In [46]:
X_train = pd.concat([X_train_num, X_train_cat], axis=1)
X_test = pd.concat([X_test_num, X_test_cat], axis=1)

In [47]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(
    class_weight='balanced',
    solver='liblinear',
    random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [48]:
from sklearn.metrics import classification_report

report_dict = classification_report(y_test, y_pred, output_dict=True)

# Dictionary to DataFrame
report_df = pd.DataFrame(report_dict).transpose()

print(report_df)

              precision  recall  f1-score  support
No                  1.0     1.0       1.0   6703.0
Yes                 1.0     1.0       1.0   1763.0
accuracy            1.0     1.0       1.0      1.0
macro avg           1.0     1.0       1.0   8466.0
weighted avg        1.0     1.0       1.0   8466.0


## Bисновки
Зважаючи на результати нової моделі, які показують ідеальні значення precision, recall і F1-score для обох класів, можна зробити наступні висновки:

Нова модель проявляє дуже високу точність у передбаченні як відсутності дощу, так і його наявності. Це може свідчити про потенційні проблеми, такі як перенавчання або виток даних, якщо ці результати не відповідають усім випробувальним даним чи зовнішнім перевіркам.

Порівнявши з результатами попередньої моделі з точністю 51% і чутливістю 76%, нова модель видається демонструвати значне поліпшення у всіх аспектах ефективності класифікації.

Вибираючи різні значення параметра solver для моделі логістичної регресії, я отримав наступні результати:

Для lbfgs, liblinear, newton-cg, newton-cholesky час виконання і результати моделі були ідентичні.

Проте, при використанні sag або saga, виникла помилка. ConvergenceWarning: The max_iter was reached which means the coef_ did not converge. and it takes a lot of time.

ConvergenceWarning свідчить про те, що модель логістичної регресії не змогла збігтися до оптимальних коефіцієнтів у задану кількість ітерацій.

Якщо я збільшую значення параметра max_iter до 1000, то досягаю тих самих результатів, але час виконання значно збільшується. Це може бути зумовлено складністю обробки даних або великою кількістю спостережень, що впливає на швидкість збігання моделі.

Рекомендується враховувати цей аспект під час вибору методу оптимізації для логістичної регресії.